In [50]:
import pandas as pd
import pyodbc
import logging
import math


In [159]:
# Leitura do arquivo CSV com pandas
df = pd.read_csv('BASE_MENSAL.csv', sep=";")

C:\Users\rwmac\AppData\Local\Temp\ipykernel_2788\3411498283.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('BASE_MENSAL.csv', sep=";")


In [160]:
# Substituir a string "NUUL" por valores nulos (None)
df.replace('NULL', None, inplace=True)

In [163]:
# Converter a data no formato adequado, definindo dayfirst=True para interpretar dd/mm/aaaa
df['dt_conce'] = pd.to_datetime(df['dt_conce'], errors='coerce', dayfirst=True)
df['dt_mov'] = pd.to_datetime(df['dt_mov'], errors='coerce', dayfirst=True)


In [169]:
colunas=df.columns

In [171]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 34 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   ctr                     100000 non-null  object        
 1   co_ope                  100000 non-null  int64         
 2   cpf_cnpj                100000 non-null  object        
 3   ic_caixa                100000 non-null  int64         
 4   tp_pessoa               100000 non-null  int64         
 5   co_sis                  100000 non-null  int64         
 6   unidade                 100000 non-null  int64         
 7   co_ag_relac             100000 non-null  int64         
 8   dt_conce                100000 non-null  datetime64[ns]
 9   DD_VENCIMENTO_CONTRATO  1289 non-null    float64       
 10  vlr_conce               100000 non-null  object        
 11  posicao                 100000 non-null  int64         
 12  da_ini                  100000 

In [120]:
df.head()

,ctr,co_ope,cpf_cnpj,ic_caixa,tp_pessoa,co_sis,unidade,co_ag_relac,dt_conce,DD_VENCIMENTO_CONTRATO,...,co_segger,co_segger_gp,co_segad,rat_h5,rat_h6,ic_atacado,ic_reg,ic_rj,ic_honrado,am_honrado
0,C12C9D8B-0A10-4239-8B0D-A0962CFA02D2,2195,1AFF1F54-ADE6-4FB9-808C-54F4E14A2A92,1,1,3196,3392,3392,2023-05-17,NaN,...,3,11,FV4,NaN,NaN,0,0,0,0,NaN
1,F73EAEEE-F939-45F1-902B-1B6833127851,2195,4550C543-1E3A-4BBA-B9D7-AD8929CD71B5,1,1,3196,1623,1623,2023-06-09,NaN,...,3,11,FV3,NaN,NaN,0,0,0,0,NaN
2,20DBFC8B-AB57-48E7-9B64-31F6697C8B4E,2195,EB48AD46-8DA5-4E71-A24C-E4F0A4C3DFA3,1,1,3196,24,24,2023-06-23,NaN,...,3,11,FV3,NaN,NaN,0,0,0,0,NaN
3,DCCAB84E-D999-46E2-A6B7-EA2C6A0ECC26,2195,D4D1AEE3-8BD5-406E-B421-B291707D60A9,1,1,3196,511,511,2023-05-17,NaN,...,3,11,FV4,NaN,NaN,0,0,0,0,NaN
4,58952E5B-2FB8-4A8F-9E54-F5F49A48753C,2195,91D8052C-B717-4AA6-8753-7D17D6FCC1C7,1,1,3196,2871,2871,2023-05-17,NaN,...,3,11,HV5,NaN,NaN,0,0,0,0,NaN


In [122]:
colunas

Index(['ctr', 'co_ope', 'cpf_cnpj', 'ic_caixa', 'tp_pessoa', 'co_sis',
       'unidade', 'co_ag_relac', 'dt_conce', 'DD_VENCIMENTO_CONTRATO',
       'vlr_conce', 'posicao', 'da_ini', 'dt_mov', 'da_atual',
       'nu_tabela_atual', 'base_calculo', 'rat_prov', 'rat_hh', 'co_mod',
       'cart', 'co_cart', 'co_seg', 'no_seg', 'co_segger', 'co_segger_gp',
       'co_segad', 'rat_h5', 'rat_h6', 'ic_atacado', 'ic_reg', 'ic_rj',
       'ic_honrado', 'am_honrado'],
      dtype='object')

# Verifique se a tabela BaseMensalTemp existe, caso contrário crie
create_table_query = '''
CREATE TABLE BaseMensalTemp (
    ctr                VARCHAR(255) NULL,
    co_ope             CHAR(4) NULL,
    cpf_cnpj           VARCHAR(255) NULL,
    ic_caixa           INT NULL,
    tp_pessoa          CHAR(4) NULL,
    co_sis             CHAR(4) NULL,
    unidade            CHAR(4) NULL,
    co_ag_relac        CHAR(4) NULL,
    dt_conce           DATE NULL,
    DD_VENCIMENTO_CONTRATO INT NULL,
    vlr_conce          DECIMAL(18, 2) NULL,
    posicao            INT NULL,
    da_ini             INT NULL,
    dt_mov             DATE NULL,
    da_atual           INT NULL,
    nu_tabela_atual    INT NULL,
    base_calculo       DECIMAL(18, 2) NULL,
    rat_prov           VARCHAR(255) NULL,
    rat_hh             BIT NULL,         
    co_mod             VARCHAR(255) NULL,
    cart               VARCHAR(255) NULL,
    co_cart            CHAR(4) NULL,     
    co_seg             INT NULL,
    no_seg             VARCHAR(6) NULL,  
    co_segger          INT NULL,
    co_segger_gp       INT NULL,
    co_segad           VARCHAR(3) NULL,  
    rat_h5             BIT NULL,         
    rat_h6             BIT NULL,         
    ic_atacado         BIT NULL,         
    ic_reg             BIT NULL,         
    ic_rj              BIT NULL,         
    ic_honrado         BIT NULL,         
    am_honrado         BIT NULL
);
'''

# Execute a criação da tabela se não existir
cursor.execute(create_table_query)
conn.commit()
print("Tabela BaseMensalTemp criada com sucesso")


In [149]:
#! pip install pandas sqlalchemy pyodbc


In [185]:
# Função para tratar valores numéricos, substituindo vírgula por ponto
def tratar_valores_numericos(valor):
    try:
        # Substituir vírgula por ponto, se houver, e converter para float
        return round(float(valor.replace(',', '.')), 2)
    except (ValueError, AttributeError):
        return None  # Substitui valores inválidos por None


In [187]:
# Aplicar o tratamento para as colunas numéricas
df['vlr_conce'] = df['vlr_conce'].apply(tratar_valores_numericos)
df['base_calculo'] = df['base_calculo'].apply(tratar_valores_numericos)

In [189]:
import pandas as pd
from sqlalchemy import create_engine


# Configuração da conexão com o SQL Server usando SQLAlchemy
server = 'RENATSPC'
database = 'ADIMPLENCIA'
driver = 'ODBC Driver 17 for SQL Server'  # Certifique-se de que esse driver está instalado

conn_string = f'mssql+pyodbc://@{server}/{database}?driver={driver}&trusted_connection=yes'

# Criando a engine de conexão
engine = create_engine(conn_string)

# Inserindo o DataFrame no SQL Server (ajustar o nome da tabela se necessário)
try:
    df.to_sql('BaseMensalTemp', con=engine, if_exists='append', index=False)
    print("Dados inseridos com sucesso.")
except Exception as e:
    print(f"Erro durante a inserção: {e}")

Dados inseridos com sucesso.
